### DETAILED EXPLANATION

This script generates blurred versions of each image in the dataset using three types of blur:

**Why Blurring Images?**

Blurring is a common type of image distortion that can occur due to various factors:

- Camera motion during capture (Motion Blur)
- Out-of-focus regions due to depth of field (Gaussian Blur)
- Uniform blurring due to poor image processing or resizing (Box Blur)

These blurs simulate real-world distortions, making our analysis more realistic.

**Types of Blur Used**

1. Gaussian Blur:
   - This blur applies a Gaussian kernel to each pixel, which results in a smooth, out-of-focus effect.
   - The blur is strongest at the center and gradually fades out (bell curve distribution).
   - We chose sigma range (0.5 to 4.0) because it provides visible but controlled blur.
   - Sigma defines the standard deviation of the Gaussian distribution (higher = stronger blur).

2. Motion Blur:
   - Simulates the effect of movement, either from a moving object or a moving camera.
   - The blur has a clear direction (angle) and intensity (length).
   - We chose length (3 to 30) to allow visible motion but not too extreme.
   - Angle (0 to 360) provides all possible directions of movement.

3. Box Blur:
   - Applies a simple averaging of pixel values in a square region around each pixel.
   - This blur is fast and commonly used for quick blur effects.
   - We chose kernel size (3 to 15) because it provides a visible blur without being too extreme.
   - The kernel size defines the square region used for averaging (higher = stronger blur).

**Why Save Parameters in a Parquet File?**

- Parquet is optimized for speed and storage efficiency.
- It is suitable for large datasets, ensuring fast read and write operations.
- Parquet allows us to store the blur parameters alongside the image identifiers, making it easy to track and analyze.

In [1]:
import numpy as np
import cv2

def gen_gaussian_kernel(size, sigma):
    """Generate a Gaussian blur kernel."""
    k1d = cv2.getGaussianKernel(size, sigma)
    kernel = k1d @ k1d.T
    return kernel

def gen_motion_kernel(length, angle):
    """Generate a motion blur kernel based on length and angle."""
    if length % 2 == 0:
        length += 1

    kernel = np.zeros((length, length), dtype=np.float32)
    center = (length - 1) / 2.0

    angle_rad = np.deg2rad(angle)
    dx = length / 2.0 * np.cos(angle_rad)
    dy = length / 2.0 * np.sin(angle_rad)

    start_point = (int(center - dx), int(center - dy))
    end_point = (int(center + dx), int(center + dy))
    
    cv2.line(kernel, start_point, end_point, 1, 1, lineType=cv2.LINE_AA)
    kernel /= np.sum(kernel)
    return kernel

def gen_box_kernel(size):
    """Generate a box blur kernel."""
    return np.ones((size, size), dtype=np.float32) / (size * size)

In [2]:
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

from common.config import TEST_ORIGINAL_DIR, TEST_BLURRED_BOX_DIR, TEST_BLURRED_GAUSSIAN_DIR, TEST_BLURRED_MOTION_DIR, BLUR_PARAM_RANGES

# Parametri per il sample: None per tutte le immagini, oppure un intero per un subset
SAMPLE_SIZE = None

image_files = sorted(list((TEST_ORIGINAL_DIR / "00000").glob("*.png")))

if SAMPLE_SIZE is not None:
    image_files = image_files[:SAMPLE_SIZE]

for blur_type in [TEST_BLURRED_BOX_DIR, TEST_BLURRED_GAUSSIAN_DIR, TEST_BLURRED_MOTION_DIR]:
    blur_type.mkdir(parents=True, exist_ok=True)

blurred_images = []

for img_path in tqdm(image_files):
    image = cv2.imread(str(img_path))
    if image is None:
        continue
    original_name = img_path.name

    blur_params = {
        "key": original_name.replace('.png', '')
    }

    # box
    kernel_size = random.randint(*BLUR_PARAM_RANGES['box']['kernel_size'])
    
    kernel = gen_box_kernel(kernel_size)
    blurred = cv2.filter2D(image, -1, kernel)
    
    blur_params['box_size'] = kernel_size

    cv2.imwrite(str(TEST_BLURRED_BOX_DIR / original_name), blurred)

    # gaussian
    sigma = random.uniform(*BLUR_PARAM_RANGES['gaussian']['sigma'])

    min_size, max_size = BLUR_PARAM_RANGES['gaussian']['size']
    ksize = int(round(6 * sigma)) | 1
    if ksize < min_size:
        ksize = min_size | 1
    elif ksize > max_size:
        ksize = max_size if max_size % 2 == 1 else max_size - 1

    kernel = gen_gaussian_kernel(ksize, sigma)
    blurred = cv2.filter2D(image, -1, kernel)

    blur_params['gaussian_sigma'] = sigma
    blur_params['gaussian_size'] = ksize
    
    cv2.imwrite(str(TEST_BLURRED_GAUSSIAN_DIR / original_name), blurred)

    # motion
    length = random.randint(*BLUR_PARAM_RANGES['motion']['length'])
    angle = random.uniform(*BLUR_PARAM_RANGES['motion']['angle'])

    kernel = gen_motion_kernel(length, angle)
    blurred = cv2.filter2D(image, -1, kernel)
    
    blur_params['motion_length'] = length
    blur_params['motion_angle'] = angle
    
    cv2.imwrite(str(TEST_BLURRED_MOTION_DIR / original_name), blurred)
    
    blurred_images.append(blur_params)

df_blur = pd.DataFrame(blurred_images)
# df_blur.to_parquet(str(const.DIR_DATASET_PATH / "blur_metadata.parquet"), index=False)

print("Blur generation completed.")

100%|██████████| 950/950 [05:42<00:00,  2.77it/s]

Blur generation completed.


In [3]:
df_blur

,key,box_size,gaussian_sigma,gaussian_size,motion_length,motion_angle
0,000000000,5,2.410632,15,17,24.686060
1,000000002,15,1.036473,7,11,343.030087
2,000000003,7,2.267535,15,30,130.346099
3,000000004,12,3.601043,21,7,299.267364
4,000000005,14,1.192350,7,15,221.249430
...,...,...,...,...,...,...
945,000001245,14,1.761435,11,27,126.243596
946,000001246,12,1.520502,9,6,60.862579
947,000001247,14,2.383077,15,13,144.040133
948,000001248,6,3.505498,21,17,231.742405


In [4]:
from common.config import IDPA_DATASET

df_original = pd.read_parquet(IDPA_DATASET)
df_original

,url,category,key,width,height,exif,aspect_ratio,size,rms_contrast,sobel_edge_strength,canny_edge_density,box_size,gaussian_sigma,gaussian_size,motion_length,motion_angle
0,http://100500foto.com/wp-content/uploads/2016/...,people,000000291,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://2gfsl7am0og1m91u0pwpiehl.wpengine.netdn...,indoor_scene,000000987,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://411posters.com/wp-content/uploads/2011/...,poster,000000382,1300.0,1728.0,{},0.752315,3169036.0,0.373365,54.414061,0.139681,7.0,0.984782,7.0,8.0,57.893700
3,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000058,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000001,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,https://www.yamaha.com/en/musical_instrument_g...,complex,000000564,1800.0,1042.0,{},1.727447,3199285.0,0.331964,60.298111,0.093565,10.0,1.759276,11.0,6.0,13.652715
1246,https://www.yellowmaps.com/usgs/topomaps/drg24...,map,000001225,1509.0,2026.0,"{""Image Tag 0x5100"": ""0""}",0.744817,5742001.0,0.221090,99.311477,0.177919,8.0,1.573692,9.0,19.0,295.231069
1247,https://www.zappos.com/images/z/2/5/1/8/8/7/25...,furniture,000000481,1920.0,1440.0,{},1.333333,4276346.0,0.253243,53.379880,0.086103,8.0,3.210482,19.0,23.0,314.072092
1248,https://ycdn.space/h/2015/02/Capitol-Hill-Loft...,indoor_scene,000000954,1050.0,1575.0,{},0.666667,2000861.0,0.241754,64.152358,0.085650,11.0,2.048122,13.0,12.0,94.506271


In [5]:
# Merge LEFT per mantenere tutte le righe di df_original
df_merged = pd.merge(
    df_original,
    df_blur,
    on='key',
    how='left'
)
df_merged

,url,category,key,width,height,exif,aspect_ratio,size,rms_contrast,sobel_edge_strength,...,box_size_x,gaussian_sigma_x,gaussian_size_x,motion_length_x,motion_angle_x,box_size_y,gaussian_sigma_y,gaussian_size_y,motion_length_y,motion_angle_y
0,http://100500foto.com/wp-content/uploads/2016/...,people,000000291,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,http://2gfsl7am0og1m91u0pwpiehl.wpengine.netdn...,indoor_scene,000000987,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,http://411posters.com/wp-content/uploads/2011/...,poster,000000382,1300.0,1728.0,{},0.752315,3169036.0,0.373365,54.414061,...,7.0,0.984782,7.0,8.0,57.893700,12.0,1.864853,11.0,20.0,207.427439
3,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000058,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,http://RealEstateAdminImages.gabriels.net/170/...,architecture,000000001,NaN,NaN,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,https://www.yamaha.com/en/musical_instrument_g...,complex,000000564,1800.0,1042.0,{},1.727447,3199285.0,0.331964,60.298111,...,10.0,1.759276,11.0,6.0,13.652715,5.0,1.339924,9.0,11.0,252.155339
1246,https://www.yellowmaps.com/usgs/topomaps/drg24...,map,000001225,1509.0,2026.0,"{""Image Tag 0x5100"": ""0""}",0.744817,5742001.0,0.221090,99.311477,...,8.0,1.573692,9.0,19.0,295.231069,6.0,2.296524,15.0,14.0,92.401375
1247,https://www.zappos.com/images/z/2/5/1/8/8/7/25...,furniture,000000481,1920.0,1440.0,{},1.333333,4276346.0,0.253243,53.379880,...,8.0,3.210482,19.0,23.0,314.072092,11.0,1.518674,9.0,13.0,80.858597
1248,https://ycdn.space/h/2015/02/Capitol-Hill-Loft...,indoor_scene,000000954,1050.0,1575.0,{},0.666667,2000861.0,0.241754,64.152358,...,11.0,2.048122,13.0,12.0,94.506271,5.0,1.342050,9.0,17.0,290.364366


In [6]:
from common.config import IDPA_DATASET

df_merged.to_parquet(IDPA_DATASET)